## Gradient descent for minimization

***Nota:*** Los problemas de aprendizaje a menudo se presentan como problemas de optimización. Por ejemplo, el entrenamiento consiste esencialmente en encontrar el modelo que mejor se ajusta a los datos. Si la noción de "mejor" se caracteriza por una **función de pérdida**, entonces el entrenamiento se presenta como un problema de minimización porque el mejor modelo corresponde a la pérdida más baja.
Alternativamente, si la noción de "mejor" se caracteriza por una **función de verosimilitud**, entonces el entrenamiento se presenta como un problema de maximización porque el mejor modelo corresponde a la mayor verosimilitud (o probabilidad). A menos que se especifique, caracterizaremos la calidad del modelo o el ajuste mediante funciones de pérdida, lo que requerirá que realicemos una minimización.

***Loss function:*** Las funciones de pérdida miden explícitamente el ajuste de un modelo en un conjunto de datos. La mayoría de las veces, medimos la pérdida con respecto a las etiquetas verdaderas, cuantificando el error entre las etiquetas predichas y verdaderas. Por lo tanto, el mejor modelo tendrá el menor error o pérdida.
Es posible que esté familiarizado con las funciones de pérdida, como la entropía cruzada o cross entropy (para la clasificación) o el error cuadrático medio (para la regresión).

Un ejemplo de una búsqueda de este tipo es una búsqueda en cuadrícula (grid search) para la selección de parámetros durante el entrenamiento de, por ejemplo, árboles de decisión. Con la búsqueda en cuadrícula, elegimos entre muchas opciones de modelado: número de hojas, profundidad máxima del árbol, etc. de manera sistemática y exhaustiva sobre una cuadrícula de parámetros.

Otra técnica de optimización más eficaz es el **descenso de gradientes**, que utiliza información de la primera derivada, o gradientes, para guiar nuestra búsqueda.

## UNDERSTANDING AND IMPLEMENTING GRADIENT DESCENT

La información de la primera derivada se conoce como el gradiente de f(w1,w2) y es una medida de la pendiente (local) de la superficie de la función. Más importante aún, el gradiente apunta en la dirección del ascenso más empinado; es decir, moverse en la dirección del ascenso más empinado nos llevará a valores mayores de f(w1,w2).
Si queremos usar información de gradiente para encontrar los minimizadores, ¡tenemos que viajar en la dirección opuesta del gradiente! Este es precisamente el principio simple, pero altamente efectivo, detrás del gradiente descendente: continúa en la dirección del gradiente negativo y terminarás en un minimizador (local).
Podemos formalizar esta intuición en el siguiente pseudocódigo, que describe los pasos del gradiente descendente. Como se muestra, el descenso de gradiente es un procedimiento iterativo que se mueve constantemente hacia un minimizador local moviéndose en la dirección del descenso más pronunciado, es decir, el gradiente negativo:

inicializar: w_old = alguna suposición inicial, convergió = Falso # mientras no convergió:
1. calcular la dirección (d) como pendiente negativa en w_old y normalizar a la unidad de longitud
2. calcular la longitud del paso usando la búsqueda de línea (distancia, a)
3. actualice la solución: w_new = w_old + distancia * dirección = w_old + a*d
4. si el cambio entre w_new y w_old está por debajo de alguna tolerancia especificada:
converged=Verdadero, así que break
5. else establece w_new = w_old, prepárate para la próxima iteración

**NOTA:** Muchos paquetes y herramientas de optimización (p. ej., scipy.optimize) brindan funciones de búsqueda de líneas exactas y aproximadas que se pueden usar para identificar la longitud de los pasos. Alternativamente, la longitud del paso también se puede establecer de acuerdo con alguna estrategia predeterminada, a menudo por eficiencia. En el aprendizaje automático, la longitud del paso a menudo se denomina tasa de aprendizaje (***learning rate*** que es la a en el ejemplo anterior) y se representa con la letra griega eta

In [ ]:
# Gradient descent
import numpy as np
from scipy.optimize import line_search


def gradient_descent(f, g, x_init, max_iter=100, args=()): # Gradient descent requires a function f and its gradient g.
    converged = False # Initializes gradient descent to “not converged”
    n_iter = 0
    x_old, x_new = np.array(x_init), None
    descent_path = np.full((max_iter + 1, 2), fill_value=np.nan)
    descent_path[n_iter] = x_old
    while not converged:
        n_iter += 1
        gradient = -g(x_old, *args) # Computes the negative gradient
        direction = gradient / np.linalg.norm(gradient) # Normalizes gradient to unit length
        step = line_search(f, g, x_old, direction, args=args) # Computes step length using line search
        if step[0] is None: # If the line search fails, make it 1.0.
            distance = 1.0
        else:
            distance = step[0]
        x_new = x_old + distance * direction # Computes the update
        descent_path[n_iter] = x_new
        err = np.linalg.norm(x_new - x_old) # Computes the change from the previous iteration
        if err <= 1e-3 or n_iter >= max_iter:
            converged = True # Converges if change is small or maximum iterations are reached
        x_old = x_new # Gets ready for the next iteration
    return x_new, descent_path

## PROPERTIES OF GRADIENT DESCENT

**Primero**, los pasos de gradiente se vuelven cada vez más pequeños a medida que nos acercamos a uno de los minimizadores. Esto se debe a que los gradientes desaparecen en los minimizadores. Lo que es más importante, el descenso de gradiente exhibe un comportamiento en zigzag porque el gradiente no apunta al minimizador local en sí mismo; más bien, apunta en la dirección de mayor ascenso (o descenso, si es negativo).
El gradiente en un punto esencialmente captura información local, es decir, la naturaleza de la función cercana a ese punto. El descenso de gradiente encadena varios de estos pasos de gradiente para llegar a un minimizador.
Cuando el descenso del gradiente tiene que pasar por valles empinados, su tendencia a usar información local hace que rebote alrededor de las paredes del valle a medida que avanza hacia el mínimo.

**En segundo lugar**, el descenso del gradiente convergió a uno de los cuatro minimizadores locales de la función de Branin. Puede hacer que converja a un minimizador diferente cambiando la inicialización. Diferentes inicializaciones hacen que el descenso del gradiente converja en diferentes minimizadores locales. Este comportamiento puede ser familiar para aquellos que han usado el agrupamiento k-means: diferentes inicializaciones a menudo producirán diferentes agrupamientos, cada uno de los cuales es una solución local diferente. Sin embargo, desde una perspectiva de aprendizaje automático, las diferentes soluciones locales pueden demostrar el mismo comportamiento de generalización. Es decir, todos los modelos aprendidos localmente óptimos tienen un rendimiento predictivo similar. Esta situación se encuentra comúnmente con las redes neuronales y el aprendizaje profundo, razón por la cual los procedimientos de entrenamiento para muchos modelos profundos se inicializan a partir de soluciones previamente entrenadas.

Tip: Si la función es convexa o tiene forma de copa en todas partes, entonces cualquier minimizador local que identifique el descenso de gradiente siempre será un minimizador global también. Este es el caso de los modelos aprendidos por los optimizadores de máquinas de vectores de soporte (SVM). Sin embargo, muchos problemas del mundo real son típicamente
no convexas y tienen varios mínimos locales. El descenso del gradiente convergerá a uno de ellos, dependiendo de la inicialización y la forma de la función en la localidad de la suposición inicial, tal es el caso de k-means.

<img src="img/gradient_rnd.jpg">

## Gradient boosting: Gradient descent + boosting

En el aumento de gradiente, nuestro objetivo es entrenar una secuencia de alumnos débiles que se aproximen al gradiente en cada iteración. El aumento de gradiente y su sucesor, el aumento de Newton (**Newton boosting**), se consideran actualmente métodos de conjuntos de última generación (state-of-the-art) y se implementan y despliegan ampliamente para varias tareas en diversas áreas de aplicación. Primero veremos la intuición del aumento de gradiente y lo contrastaremos con otro método de aumento familiar: AdaBoost

El componente clave de los métodos de conjuntos secuenciales, como AdaBoost y el aumento de gradiente, es que tienen como objetivo entrenar un nuevo estimador débil en cada iteración para corregir los errores cometidos por el estimador débil en la iteración anterior. Sin embargo, AdaBoost y el aumento de gradiente entrenan nuevos estimadores débiles en ejemplos mal clasificados de maneras bastante diferentes:
* AdaBoost identifica los ejemplos de entrenamiento de alta prioridad al ponderarlos de manera que los ejemplos mal clasificados tengan mayor peso que los clasificados correctamente. De esta forma, Ada Boost puede decirle al algoritmo de aprendizaje base en qué ejemplos de entrenamiento debe enfocarse en la iteración actual. 
* Por el contrario, el aumento de gradiente utiliza residuos o errores (entre las etiquetas verdaderas y predichas) para decirle al algoritmo de aprendizaje base en qué ejemplos de entrenamiento debe enfocarse en la siguiente iteración. ¿Qué es exactamente un residual? Para un ejemplo de entrenamiento, es simplemente el error entre la etiqueta verdadera y la predicción correspondiente. En otras palabras, ¡El gradiente negativo de la pérdida al cuadrado es exactamente el mismo que nuestro residual! Esto significa que el gradiente de la función de pérdida es una medida de la clasificación errónea y es el residuo.

<img src="img/gradient_error.jpg">

### USO DE ALUMNOS DÉBILES PARA APROXIMAR GRADIENTES

Continuando con nuestra analogía con AdaBoost, recuerde que una vez que asigna pesos a todos los ejemplos de entrenamiento, tenemos un conjunto de datos con peso aumentado (xi, yi, Di) con i = 1, …, n, de ejemplos ponderados. Por lo tanto, capacitar a un alumno débil en AdaBoost es una instancia de un problema de clasificación ponderada. Con un algoritmo de clasificación base apropiado, AdaBoost entrena un clasificador débil.

En el aumento de gradiente, ya no tenemos pesos (Di). En su lugar, tenemos residuos (o gradientes de pérdida negativa), ri, y un conjunto de datos aumentados de residuos (xi, ri). En lugar de etiquetas de clasificación (yi = 0 o 1) y pesos de ejemplo (Di), cada ejemplo de entrenamiento ahora tiene un residuo asociado, que se puede ver como una etiqueta de valor real. Por lo tanto, capacitar a un alumno débil en el aumento de gradiente es una instancia de un problema de regresión, que requiere un algoritmo de aprendizaje básico como la regresión de árbol de decisión. Cuando se entrenan, los estimadores débiles en el aumento de gradiente se pueden ver como gradientes aproximados.

<img src="img/gradient_aprox.jpg">

Para resumir, el aumento de gradiente combina el descenso y el aumento de gradiente:

* Al igual que AdaBoost, el aumento de gradiente entrena a un alumno débil para corregir los errores cometidos por el alumno débil anterior. AdaBoost usa pesos de ejemplo para enfocar el aprendizaje en ejemplos mal clasificados, mientras que el aumento de gradiente usa residuos de ejemplo para hacer lo mismo.
* Al igual que el descenso de gradiente, el aumento de gradiente actualiza el modelo actual con información de gradiente. El descenso de gradiente usa el gradiente negativo directamente, mientras que el aumento de gradiente entrena un regresor débil sobre los residuales negativos para aproximar el gradiente.

<img src="img/gradient_boosting.jpg">

### Implementing Gradient boosting for the squared loss

In [ ]:
from scipy.optimize import minimize_scalar
from sklearn.tree import DecisionTreeRegressor


def fit_gradient_boosting(X, y, n_estimators=10):
    n_samples, n_features = X.shape # Gets dimensions of the data set
    n_estimators = 10
    estimators = [] # Initializes an empty ensemble
    F = np.full((n_samples, ), 0.0) # Predicts the ensemble on the training set
    for t in range(n_estimators):
        residuals = y - F # Computes residuals as negative gradients of the squared loss
        h = DecisionTreeRegressor(max_depth=1)
        h.fit(X, residuals) # Fits weak regression tree (h_t) to the examples and residuals
        hreg = h.predict(X) # Gets predictions of the weak learner, h_t
        loss = lambda a: np.linalg.norm(y - (F + a * hreg))**2 # Sets up the line search problem
        step = minimize_scalar(loss, method='golden') # Finds the best step length using the golden section search
        a = step.x
        F += a * hreg # Updates the ensemble predictions
        estimators.append((a, h)) # Updates the ensemble
    return estimators

Predictions using gradient-boosted model

In [ ]:
def predict_gradient_boosting(X, estimators):
    pred = np.zeros((X.shape[0], )) # Initializes all the predictions to 0
    for a, h in estimators:
        pred += a * h.predict(X) # Aggregates individual predictions from each regressor
    y = np.sign(pred) # Converts weighted predictions to –1/1 labels
    return y

In [ ]:
# EXAMPLE
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.15, random_state=13)
y = 2 * y - 1 # Converts training labels to –1/1

from sklearn.model_selection import train_test_split

Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25, random_state=11) # Splits into train and test sets
estimators = fit_gradient_boosting(Xtrn, ytrn)
ypred = predict_gradient_boosting(Xtst, estimators)

from sklearn.metrics import accuracy_score

tst_err = 1 - accuracy_score(ytst, ypred) # Trains and gets the test error
tst_err

### Gradient boosting with scikit-learn

*NOTA:* Entrenar un GradientBoostingClassifier con la pérdida exponencial es muy similar (pero no exactamente igual) a entrenar un AdaBoostClassifier.

In [ ]:
# EXAMPLE
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True)
# Loads the data set and splits it into training and test sets
Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25, random_state=13)

from sklearn.ensemble import GradientBoostingClassifier

ensemble = GradientBoostingClassifier(
    max_depth=1, # Trains a gradient boosting model with these learning parameters
    n_estimators=20,
    learning_rate=0.75
    )
ensemble.fit(Xtrn, ytrn)

ypred = ensemble.predict(Xtst)
err = 1 - accuracy_score(ytst, ypred)
print(err)

### Histogram-based gradient boosting

Imagina que tienes un conjunto de datos que contiene información sobre diferentes características de ciertos objetos. Por ejemplo, podrías tener datos sobre el tamaño, el color y la forma de diferentes frutas. El histogram-based tree learning tomaría estos datos y los dividiría en "contenedores" o "bins" en función de los valores de las características. Por ejemplo, podrías tener un contenedor para las frutas grandes y otro para las frutas pequeñas, y así sucesivamente.

Una vez que los datos se han organizado en histogramas, se pueden construir árboles de decisión para tomar decisiones predictivas. Un árbol de decisión es una estructura en forma de árbol donde cada nodo representa una pregunta sobre las características de los datos. Por ejemplo, un nodo podría preguntar si una fruta es grande o pequeña. Dependiendo de la respuesta, el árbol se ramificaría hacia diferentes nodos con preguntas más específicas.

El histogram-based tree learning utiliza estos árboles de decisión para hacer predicciones. Cuando se presenta una nueva muestra de datos, el árbol recorre los nodos y toma decisiones basadas en las características de la muestra. Al final del recorrido, el árbol proporciona una predicción sobre la clase o valor objetivo de la muestra.

Una de las ventajas del histogram-based tree learning es que permite un procesamiento más eficiente de grandes conjuntos de datos, ya que los histogramas resumen la información de manera compacta. Además, al utilizar árboles de decisión, el método es fácilmente interpretable y permite entender cómo se toman las decisiones.

### HistogramGradientBoostingClassifier with sckit-learn

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier


ensemble = HistGradientBoostingClassifier(
    max_depth=2, 
    # Initializes a histogram-based gradient-boosting classifier
    max_iter=20,
    learning_rate=0.75
    )
ensemble.fit(Xtrn, ytrn) # Trains the ensemble

ypred = ensemble.predict(Xtst)
err = 1 - accuracy_score(ytst, ypred)
print(err)

## LightGBM: A framework for gradient boosting

Light Gradient Boosted Machines (LightGBM) es un marco de mejora de gradiente de código abierto que fue desarrollado y lanzado originalmente por Microsoft. En esencia, LightGBM es esencialmente un enfoque de aumento de gradiente basado en histogramas. Sin embargo, también tiene varias características algorítmicas y de modelado que le permiten manejar datos a gran escala. En particular, LightGBM ofrece las siguientes ventajas:

* Aceleraciones algorítmicas, como el muestreo unilateral basado en gradientes y la agrupación de características exclusivas que dan como resultado un entrenamiento más rápido y un menor uso de memoria.
* Compatibilidad con una gran cantidad de funciones de pérdida para clasificación, regresión y clasificación, así como funciones de pérdida personalizadas específicas de la aplicación.
* Compatibilidad con el aprendizaje en GPU y en paralelo, lo que permite que LightGBM maneje conjuntos de datos a gran escala.

Gradient-based One-Side Sampling (GOSS) procedure.

Brevemente, GOSS realiza los siguientes pasos:

1. Use la magnitud del gradiente, similar a AdaBoost, que usa pesos de muestra.
Recuerde que el gradiente indica cuánto más se puede mejorar la predicción: los ejemplos bien entrenados tienen gradientes pequeños, mientras que los ejemplos poco entrenados (por lo general, mal clasificados o confusos) tienen gradientes grandes.
2. Seleccione el a% superior de los ejemplos con los gradientes más grandes; llame a este subconjunto top.
3. Muestra aleatoriamente b% de los ejemplos restantes; llame a este subconjunto rand.
4. Asigne pesos a los ejemplos en ambos conjuntos: w_top = 1, w_rand = (100 - a) / b
5. Entrene un regresor base sobre estos datos muestreados (ejemplos, residuos, pesos).

Los pesos calculados en el paso 4 aseguran que haya un buen equilibrio entre las muestras poco entrenadas y bien entrenadas. En general, dicho muestreo también fomenta la diversidad de conjuntos, lo que en última instancia conduce a mejores conjuntos.

EXCLUSIVE FEATURE BUNDLING

Además de una gran cantidad de ejemplos de capacitación, los macrodatos también suelen presentar el desafío de una dimensionalidad muy alta, que puede afectar negativamente la construcción de histogramas y ralentizar el proceso de capacitación general. De manera similar a los ejemplos de entrenamiento de reducción de muestreo, si también podemos reducir la muestra de las características, es posible obtener mejoras (a veces muy grandes) en la velocidad de entrenamiento. Esto es así especialmente cuando el espacio de características es escaso y las características son mutuamente excluyentes.

Un ejemplo común de un espacio de características de este tipo es cuando aplicamos vectorización one-hot a variables categóricas. Por ejemplo, considere una variable categórica que toma 10 valores únicos. Cuando se vectoriza one-hot, esta variable se expande a 10 variables binarias, de las cuales solo una es distinta de cero y todas las demás son cero. Esto hace que las 10 columnas correspondientes a esta característica sean muy escasas.

Exclusive Feature Bundling (EFB) funciona a la inversa, explota esta escasez y tiene como objetivo comprimir columnas mutuamente excluyentes en una sola columna para reducir la cantidad de funciones efectivas. A un alto nivel, EFB realiza dos pasos:
1. Identifique las características que se pueden agrupar midiendo los conflictos o la cantidad de veces que ambas características son distintas de cero simultáneamente. La intuición aquí es que si dos características a menudo se excluyen mutuamente, tienen poco conflicto y se pueden agrupar.
2. Combine las características de bajo conflicto identificadas en un paquete de características. La idea aquí es preservar la información con cuidado al fusionar valores distintos de cero, lo que generalmente se hace agregando compensaciones a los valores de las características para evitar superposiciones.

Al fusionar funciones de esta manera, EFB reduce de manera efectiva la cantidad total de funciones, lo que a menudo hace que el entrenamiento sea mucho más rápido.

## Gradient boosting with LightGBM

In [ ]:
# EXAMPLE
from lightgbm import LGBMClassifier


gbm = LGBMClassifier(
    boosting_type='gbdt',   # 'rf' — Trains traditional random forest ensembles
                            # 'gbdt' — Trains an ensemble using traditional gradient boosting
                            # 'goss' — Trains an ensemble using GOSS
                            # 'dart' — Trains an ensemble using Dropouts meet Multiple Additive Regression Trees
    n_estimators=20, 
    max_depth=1)
gbm.fit(Xtrn, ytrn)

from sklearn.metrics import accuracy_score

ypred = gbm.predict(Xtst)
accuracy_score(ytst, ypred)

### Cross validation with LightGBM and scikit-learn

LightGBM funciona muy bien con scikit-learn y podemos combinar las funcionalidades relevantes de ambos paquetes para realizar el aprendizaje modelo.Combinamos la clase StratifiedKFold de scikit-learn para dividir los datos de entrenamiento en 10 pliegues de conjuntos de entrenamiento y validación. StratifiedKFold asegura que preservamos las distribuciones de clases, es decir, las fracciones de diferentes clases en los pliegues. Una vez que se configuran los pliegues de CV, podemos entrenar y validar modelos en estos 10 pliegues para diferentes opciones de tasas de aprendizaje: 0.1, 0.2, …, 1.0.

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np


n_learning_rate_steps, n_folds = 10, 10 # Initializes learning rates and number of cross validation folds
learning_rates = np.linspace(0.1, 1.0, num=n_learning_rate_steps)
splitter = StratifiedKFold( # Splits data into training and validation folds
    n_splits=n_folds, 
    shuffle=True, 
    random_state=42)
trn_err = np.zeros((n_learning_rate_steps, n_folds)) # Saves training and
val_err = np.zeros((n_learning_rate_steps, n_folds)) # validation errors
for i, rate in enumerate(learning_rates): # Trains a LightGBM classifier for each fold with different learning rates
    for j, (trn, val) in enumerate(splitter.split(X, y)):
        gbm = LGBMClassifier(
            boosting_type='gbdt', 
            n_estimators=10, 
            max_depth=1, 
            learning_rate=rate)
        gbm.fit(X[trn, :], y[trn])
        trn_err[i, j] = (1 - accuracy_score(y[trn], gbm.predict(X[trn, :]))) * 100 # Saves training and
        val_err[i, j] = (1 - accuracy_score(y[val], gbm.predict(X[val, :]))) * 100 # validation errors
trn_err = np.mean(trn_err, axis=1) # Averages training and
val_err = np.mean(val_err, axis=1) # validation errors across folds

### Cross validation with LightGBM

LightGBM proporciona su propia funcionalidad para realizar CV con opciones de parámetros dadas a través de una función llamada cv

In [ ]:
from lightgbm import cv, Dataset


trn_data = Dataset(Xtrn, label=ytrn) # Puts data into a LightGBM “Dataset” object

params = {'boosting_type': 'gbdt', 
          'objective': 'cross_entropy', 
          'learning_rate': 0.25, # Specifies learning parameters
          'max_depth': 1}

cv_results = cv(
    params, 
    trn_data, 
    num_boost_round=100, 
    nfold=5, # Performs 5-fold CV, each with 100 estimators
    stratified=True, 
    shuffle=True)

### Early stopping with LightGBM

Otra forma de controlar el comportamiento de sobreadaptación es detenerse antes de tiempo. Como hemos visto con AdaBoost, la idea de detenerse antes de tiempo es bastante sencilla. A medida que entrenamos conjuntos secuenciales, entrenamos un estimador base en cada iteración. Este proceso continúa hasta que alcanzamos el tamaño de conjunto especificado por el usuario (en LightGBM, hay varios alias para especificar esto: n_estimators, num_trees, num_rounds).

In [ ]:
from sklearn.model_selection import train_test_split


Xtrn, Xval, ytrn, yval = train_test_split(X, y, 
                                          test_size=0.2, # Splits data into train and validation sets
                                          shuffle=True, 
                                          random_state=42) 

# Performs early stopping if there’s no change in the validation score after five rounds
gbm = LGBMClassifier(boosting_type='gbdt', 
                     n_estimators=50, 
                     max_depth=1, 
                     early_stopping=5) 

# Uses AUC as the validation scoring metric for early stopping
gbm.fit(Xtrn, ytrn, eval_set=[(Xval, yval)], eval_metric='auc') 

**Nota:** En su salida, LightGBM informa dos métricas: AUC, que especificamos como la métrica de evaluación, y pérdida logística binaria, que es su métrica de evaluación predeterminada. Dado que especificamos la detención anticipada con respecto a AUC, el algoritmo finaliza aunque la pérdida logística binaria siga disminuyendo. Dicho de otra manera, si hubiéramos utilizado la pérdida logística binaria como nuestra métrica de evaluación, la detención temprana no habría terminado tan temprano y habría continuado. En situaciones prácticas, tales métricas a menudo dependen de la tarea y deben elegirse cuidadosamente teniendo en cuenta la aplicación posterior.

### Custom loss functions

Con el aumento de gradiente en general, y LightGBM específicamente, una vez que tenemos una función de pérdida, podemos entrenar y evaluar rápidamente los modelos que están dirigidos a nuestro problema.

**THE FOCAL LOSS**

La pérdida focal, en general, fue diseñada y es adecuada para problemas de clasificación con desequilibrios de clase. Es una modificación de la clásica pérdida de entropía cruzada que se enfoca más en los ejemplos más difíciles de clasificar, mientras ignora los ejemplos más fáciles.

**GRADIENT BOOSTING WITH THE FOCAL LOSS**

Para usar la pérdida focal para entrenar árboles de decisión potenciados por gradiente (GBDT), tenemos que proporcionar a LightGBM dos funciones:
* La propia función de pérdida real, que se utilizará para las evaluaciones de funciones y la puntuación durante el aprendizaje
* La primera derivada (gradiente) y la segunda derivada (hessiana) de la función de pérdida, que se usarán para aprender los árboles de estimación base constituyentes (LightGBM utiliza la información de Hessian para aprender en los nodos hoja)

La función *focal_loss* es la pérdida en sí misma.
La función *focal_loss_metric* convierte *focal_loss* en una métrica de puntuación para usar con LightGBM.
La función *focal_loss_objective* devuelve el gradiente y el Hessian de la función de pérdida para que LightGBM los use en el aprendizaje del árbol. Esta función tiene el sufijo poco intuitivo de "objetivo" para ser coherente con el uso de LightGBM, como se verá en breve.

In [ ]:
from scipy.misc import derivative


# Defines the focal loss function
def focal_loss(ytrue, ypred, gamma=2.0):
    p = 1 / (1 + np.exp(-ypred))
    loss = -(1 - ytrue) * p**gamma * np.log(1 - p) - ytrue * (1 - p)**gamma * np.log(p)
    return loss

# Wrapper function that returns a LightGBM-compatible scoring metric
def focal_loss_metric(ytrue, ypred): 
    return 'focal_loss_metric', np.mean(focal_loss(ytrue, ypred)), False

def focal_loss_objective(ytrue, ypred):
    func = lambda z: focal_loss(ytrue, z)
    grad = derivative(func, ypred, n=1, dx=1e-6) # Autodifferentiation computes /
    hess = derivative(func, ypred, n=2, dx=1e-6) # gradient and Hessian derivatives
    return grad, hess

**Nota:** Se debe tener cuidado para garantizar que la función de pérdida, la métrica y el objetivo sean compatibles con vectores; es decir, pueden tomar objetos similares a matrices ytrue e ypred como entradas.

In [ ]:
# EXAMPLE

# Sets objective to ensure that LightGBM uses the gradients of the focal loss for learning
gbm_focal_loss = LGBMClassifier(
    objective=focal_loss_objective, 
    learning_rate=0.25, 
    n_estimators=20, 
    max_depth=1
    )

# Sets metric to ensure that LightGBM uses the focal loss for evaluation
gbm_focal_loss.fit(
    Xtrn, 
    ytrn, 
    eval_set=[(Xval, yval)], 
    eval_metric=focal_loss_metric
    )

from scipy.special import expit # Imports the sigmoid function from “scipy”

# Gets raw scores and then computes the probability of class=1 using the sigmoid function
probs = expit(
    gbm_focal_loss.predict(Xval,raw_score=True)
    )

# Converts to a 0/1 label, where the prediction is class=1 if probability > 0.5, and class=0 otherwise
ypred = (probs > 0.5).astype(float)

accuracy_score(yval, ypred)